### Description:
This notebook is used to evaluate the quality of the features, determining whether they are suitable for use in a machine learning algorithm or if it's necessary to seek alternative sources to obtain higher-quality data.

Dataset used: https://www.kaggle.com/datasets/blastchar/telco-customer-churn

# 1. Set up:

- a) Import libraries
- b) Define features to assess and plot settings
- c) Set up folder name

#### a) Import libraries

In [ ]:
import os
import pandas as pd
import seaborn as sns
import numpy as np
import datetime
from dateutil.parser import parse
from code_fd.feature_value_calc import *
from code_fd.graphing import *

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages 

# ignore warnings
import warnings
warnings.filterwarnings('ignore')

#### b) Define features and plot settings

In [ ]:
# categorical features
cat_feature_name_list = ['gender','Partner', 'SeniorCitizen', 'Dependents', 'PhoneService',
                         'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup',
                         'DeviceProtection', 'TechSupport', 'StreamingTV', 'Contract',
                         'PaperlessBilling', 'PaymentMethod']

# continuous features
con_feature_name_list = ['tenure', 'MonthlyCharges', 'TotalCharges']

# target feature
# target feature value: the feature value for calculating proportion %
target_feature = 'Churn'
target_feature_name = 'churn'
target_feature_value = 'Yes'

# columne date in the feature table
date_feature = 'temp_date'

# plot: Y to rotate x axis values at 90 deg
rotate_x_axis = 'Y'

# plot: the y secondary axis title 
proportion_name = target_feature_name + ' proportion (%)'

#### c) Set up output folder name

In [ ]:
# today's date
TODAY = datetime.datetime.now()
DTFORMAT = '%Y%m%d'

# name of the folder, excel file and pdfs
folder_path = 'output_sample/telco_churn_feature_diagnostics'
file_name = 'summary_table_' + TODAY.strftime(DTFORMAT)
pdf_null_fn = 'plot_missing_value_' + TODAY.strftime(DTFORMAT)
pdf_count_fn = 'plot_feature_value_count_and_' + target_feature_name + '_proportion_' + TODAY.strftime(DTFORMAT)
pdf_count_month_fn = 'plot_feature_value_count_by_month_' + TODAY.strftime(DTFORMAT)

In [ ]:
# create new folder
check_folder = os.path.isdir(folder_path)

if not check_folder:
    os.mkdir(folder_path)

# 2. Read data set:

- a) Import dataset
- b) Data preparation

#### a) Import data set

In [ ]:
df = pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv")

# assuming churn rate recorded on 2023-12-31
df['temp_date'] = datetime.datetime(2023, 12, 31)

#### b) Data preparation

In [ ]:
# add month year into the data
df['monthyear'] = df[date_feature].dt.year.astype(str) + df[date_feature].dt.month.map("{:02}".format).astype(str)

# replace null target feature to NA
df[target_feature] = df[target_feature].fillna('NA')

# convert target feature to string
df[target_feature] = df[target_feature].astype(str)

# convert categorical features to string 
df[cat_feature_name_list] = df[cat_feature_name_list].astype(str)
df[cat_feature_name_list] = df[cat_feature_name_list].replace('nan', np.nan)
df[cat_feature_name_list] = df[cat_feature_name_list].replace('None', np.nan)

# convert continuous features to float
df[con_feature_name_list] = df[con_feature_name_list].replace(r'^\s*$', np.nan , regex=True)
df[con_feature_name_list] = df[con_feature_name_list].astype(float)

# shape
tot_row, tot_col = df.shape

# 3. Data Quality assessment

- a) Summary statistics 
- b) Feature count (by monthyear and target feature)
- c) Missing value count

#### a) Summary statistics 

In [ ]:
# concatenate list of feature
feature_name_list = cat_feature_name_list + con_feature_name_list

In [ ]:
# get summary stats for target feature, categorical and continuous features
target_ds = get_summary_statistics('Target', [target_feature], df)
cat_ds = get_summary_statistics('Categorical', cat_feature_name_list, df)
con_ds = get_summary_statistics('Continuous', con_feature_name_list, df)

#### b) Feature count

In [ ]:
# count by feature value, target feature, and monthyear
cat_feature_value_df = get_feature_value_count_categorical(cat_feature_name_list, df, target_feature)
con_feature_value_df = get_feature_value_count_continuous(con_feature_name_list, df, target_feature)

# count by feature value only
cat_df_feature_count = count_by_feature_value(cat_feature_value_df, '', '')
con_df_feature_count = count_by_feature_value(con_feature_value_df, '', '')

# count by feature value and monthyear
cat_df_feature_month_count = count_by_feature_value(cat_feature_value_df, 'monthyear', '')
con_df_feature_month_count = count_by_feature_value(con_feature_value_df, 'monthyear', '')
target_feature_month_count = count_target_by_month(df, target_feature)

# [Continuous only] Median value by month
con_df_median_month = get_median_value_continuous_feature(con_feature_name_list, df)

# Count by feature value and target feature
cat_df_feature_target_count = count_by_feature_value_target_feature(cat_feature_value_df, target_feature)
con_df_feature_target_count = count_by_feature_value_target_feature(con_feature_value_df, target_feature)

#### c) Missing value count

In [ ]:
# count null values by feature name, feature value, month for target, categorical and continuous features
cat_df_feature_month_nulls_count = count_by_month_missing_values(cat_df_feature_month_count, 'NA')
con_df_feature_month_nulls_count = count_by_month_missing_values(con_df_feature_month_count, 'NA')
target_feature_month_nulls_count = count_by_month_missing_values(target_feature_month_count, 'NA')

# 4. Graphing

- a) Plot missing values
- b) Plot feature value and target feature value proportion
- c) Plot feature count plot

#### a) Plot missing values

In [ ]:
# create pdf pages
pp = PdfPages(folder_path + '/' + pdf_null_fn + '.pdf')

In [ ]:
# sort by month and year
cat_df_feature_month_nulls_count = cat_df_feature_month_nulls_count.sort_values(by='monthyear', ascending=True, ignore_index=True)
con_df_feature_month_nulls_count = con_df_feature_month_nulls_count.sort_values(by='monthyear', ascending=True, ignore_index=True)
target_feature_month_nulls_count = target_feature_month_nulls_count.sort_values(by='monthyear', ascending=True, ignore_index=True)

# append missing values count
df_nulls_count = pd.concat([cat_df_feature_month_nulls_count, con_df_feature_month_nulls_count], ignore_index=True)

# plot missing values by monthyear
plot_null_by_month(feature_name_list, df_nulls_count, rotate_x_axis, pp)
plot_target_feature_null_by_month(target_feature_month_nulls_count, target_feature, rotate_x_axis, pp)

# close pdf
pp.close()

#### b) Plot Feature value and target feature 

In [ ]:
# create pdf pages
pp = PdfPages(folder_path + '/' + pdf_count_fn + '.pdf')

In [ ]:
# append count
df_feature_count = pd.concat([cat_df_feature_count, con_df_feature_count], ignore_index=True)

# append count by feature value and target feature proportion
df_feature_target_count = pd.concat([cat_df_feature_target_count, con_df_feature_target_count], ignore_index=True)
df_feature_target_count_proportion = df_feature_target_count[df_feature_target_count[target_feature] == target_feature_value]
df_feature_target_count_proportion.rename(columns={'proportion (%)': proportion_name}, inplace=True)

# ensure feature value is string
df_feature_target_count_proportion['feature value'] = df_feature_target_count_proportion['feature value'].astype(str)

# plot bar chart of count by feature value with proportion as secondary axis
plot_count_by_feature_value(feature_name_list, df_feature_count, df_feature_target_count_proportion, proportion_name, rotate_x_axis, pp)

# close pdf
pp.close()

#### c) Plot feature count

In [ ]:
# create pdf pages
pp = PdfPages(folder_path + '/' + pdf_count_month_fn + '.pdf')

In [ ]:
# sort by monthyear
cat_df_feature_month_count = cat_df_feature_month_count.sort_values(by='monthyear', ascending=True, ignore_index=True) 
con_df_feature_month_count = con_df_feature_month_count.sort_values(by='monthyear', ascending=True, ignore_index=True) 
con_df_median_month = con_df_median_month.sort_values(by='monthyear', ascending=True, ignore_index=True) 

# append
df_feature_month_count = pd.concat([cat_df_feature_month_count, con_df_feature_month_count], ignore_index=True)

# remove missing values 
cat_df_feature_month_count_noNA = cat_df_feature_month_count[cat_df_feature_month_count['feature value'] != 'NA']
cat_df_feature_name_list_noNA = list(cat_df_feature_month_count_noNA['feature name'].unique())

# [Categorical] plot count of feature values by monthyear
# [Continuous] plot median against monthyear
plot_time_series_feature_values(cat_df_feature_name_list_noNA, cat_df_feature_month_count_noNA, pp)
plot_time_series_median(con_feature_name_list, con_df_median_month, pp)

# close pdf
pp.close()

# 5. Export results 

In [ ]:
# file name
file_name_excel = file_name + '.xlsx'

# summary stats
sum_stats_tbl = pd.concat([target_ds, cat_ds, con_ds], ignore_index=True)

# include target feature missing values count by month
df_nulls_count = pd.concat([df_nulls_count, target_feature_month_nulls_count], ignore_index=True)

In [ ]:
# write summary tables into excel
writer = pd.ExcelWriter(folder_path + '/' + file_name_excel, engine='xlsxwriter')
sum_stats_tbl.to_excel(writer, sheet_name='Summary_Stats', index=False)
df_feature_count.to_excel(writer, sheet_name='Count by feature value', index=False)
df_feature_month_count.to_excel(writer, sheet_name='Count by feature value_month', index=False)
df_feature_target_count.to_excel(writer, sheet_name='Count by feature value_target', index=False)
df_nulls_count.to_excel(writer, sheet_name='Null by month', index=False)
writer.close()